<a href="https://colab.research.google.com/github/iansawicki/fireworks/blob/main/notebooks/Supervised_Fine_Tuning_Text2SQL_on_Fireworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning Text2SQL on Fireworks

This notebook demonstrates the **Fireworks CLI commands** for fine-tuning with minimal Python wrapper code.

## 📋 Table of Contents

| Section | Description | Key CLI Commands |
|---------|-------------|------------------|
| **0. Setup** | Install dependencies and environment | - |
| **1. Authentication** | Setup CLI and verify credentials | `firectl signin`, `firectl whoami` |
| **2. Dataset Preparation** | Load and convert Spider dataset | - |
| **3. Dataset Upload** | Upload to Fireworks (Local + GCS) | `firectl create dataset` |
| **4. Fine-tuning** | Create and monitor training job | `firectl create supervised-fine-tuning-job` |
| **5. Deployment** | Deploy trained model | `firectl create deployment` & `firectl load-lora` |
| **6. Testing** | Query your deployed model | `curl` + Python requests |
| **7. Duck DB Demo** | Query your deployed model | Python requests + LM Created Queries|

## 🎯 What You'll Learn

- **CLI Commands**: The exact `firectl` commands to run
- **End-to-End Workflow**: From dataset to deployed model  
- **Two Approaches**: Local files vs. Google Cloud Storage
- **Best Practices**: Dataset formats, fine-tuning parameters, testing

## 📋 Prerequisites

1. **Fireworks AI Account**: Sign up at [fireworks.ai](https://fireworks.ai)
2. **Fireworks CLI (mac)**: `brew tap fw-ai/firectl && brew install firectl`
3. **API Key**: `export FIREWORKS_API_KEY='your-key'`
4. **Optional**: HuggingFace token for private datasets

## 🚀 Quick Start

If you just want the commands, here's the complete workflow:

```bash
# 1. Setup
export FIREWORKS_API_KEY='your-key'
firectl signin

# 2. Upload dataset (choose one)
firectl create dataset my-dataset ./data.jsonl                    # Local file
firectl create dataset my-dataset --external-url gs://bucket/data.jsonl  # GCS

# 3. Start fine-tuning
firectl create supervised-fine-tuning-job \
  --dataset my-dataset --output-model my-model \
  --base-model accounts/fireworks/models/qwen2p5-coder-32b-instruct \
  --epochs 3 --turbo --early-stop --eval-auto-carveout

# 4. Monitor and deploy
firectl get supervised-fine-tuning-job JOB_ID
firectl create deployment accounts/ACCOUNT/models/my-model --enable-addons
```

# 1. Authentication & Environment Setup

## 📋 Section Overview
- Install Fireworks CLI
- Set API key and authenticate
- Verify setup with Python
- Optional: Setup HuggingFace access

## 🔧 Required CLI Commands

Run these commands in your terminal **before** running this notebook:

```bash
# 1. Install Fireworks CLI (on mac)
brew tap fw-ai/firectl && brew install firectl

# Or Linux (If you're using Collab, do this in the terminal.)
wget -O firectl.gz https://storage.googleapis.com/fireworks-public/firectl/stable/linux-amd64.gz
gunzip firectl.gz
sudo install -o root -g root -m 0755 firectl /usr/local/bin/firectl

# 2. Sign into Fireworks
firectl signin

# 3. Verify authentication
firectl whoami

# 4. Set your HF API key
export HUGGINGFACE_HUB_TOKEN='<your-hf-token-here>

# 5. Set your API key
export FIREWORKS_API_KEY='<your-api-key-here>'
```

**Important**: The Python code below will check if these steps were completed successfully.

In [ ]:
%%writefile requirements.txt
datasets>=2.14.0,<2.16.0
huggingface-hub>=0.17.0,<0.20.0
requests==2.32.3
pandas==2.2.2
pyarrow>=14.0.0,<16.0.0
fsspec>=2023.1.0,<2024.0.0

In [ ]:
# Install notebook dependencies
!pip cache purge
!pip install -r requirements.txt

In [ ]:
# Install Fireworks CLI (on linux)
! wget -O firectl.gz https://storage.googleapis.com/fireworks-public/firectl/stable/linux-amd64.gz
! gunzip firectl.gz
! sudo install -o root -g root -m 0755 firectl /usr/local/bin/firectl

# Sign into fireworks firectl CLI
! firectl signin

In [ ]:
import subprocess, sys, os
import getpass
import os

# Use getpass to get the API keys securely
FIREWORKS_API_KEY = getpass.getpass('Enter your Fireworks API Key: ')
HUGGINGFACE_HUB_TOKEN = getpass.getpass('Enter your HuggingFace Hub Token (optional): ')

# Set the environment variables
os.environ["FIREWORKS_API_KEY"] = FIREWORKS_API_KEY
os.environ["HUGGINGFACE_HUB_TOKEN"] = HUGGINGFACE_HUB_TOKEN

print("API keys set securely.")

In [ ]:
# Setup HuggingFace Authentication (Optional)

import os
from huggingface_hub import login

print("🤗 Setting up HuggingFace authentication...")

hf_token = HUGGINGFACE_HUB_TOKEN
if hf_token:
    login(token=hf_token)
    print("✅ Logged into HuggingFace!")
else:
    print("⚠️  No HUGGINGFACE_HUB_TOKEN found (this is optional for public datasets)")
    print("💡 For private datasets, get a token from: https://huggingface.co/settings/tokens")
    print("💡 Then set: export HUGGINGFACE_HUB_TOKEN='your_token_here'")

## ✅ Verify Setup

This Python code checks if the CLI commands above were successful:

In [ ]:
# Check Prerequisites
import subprocess
import os

print("🔧 Checking Prerequisites...")

# Check API key
api_key = FIREWORKS_API_KEY
if api_key:
    print(f"✅ API Key found: {api_key[:10]}...")
else:
    print("❌ No FIREWORKS_API_KEY found")
    print("💡 Set it with: export FIREWORKS_API_KEY='your-key'")

# Check firectl
try:
    result = subprocess.run(["firectl", "version"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ firectl installed: {result.stdout.strip()}")
    else:
        print("❌ firectl not working")
except FileNotFoundError:
    print("❌ firectl not found")
    print("💡 Install with: brew tap fw-ai/firectl && brew install firectl")

# Check if signed in
try:
    result = subprocess.run(["firectl", "whoami"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Signed into Fireworks CLI")
        print(result.stdout.strip())
    else:
        print("⚠️  Not signed into Fireworks CLI")
        print("💡 Sign in with: firectl signin")
except:
    print("⚠️  Could not check Fireworks CLI status")

print("🚀 Prerequisites check complete!")


# 2. Dataset Preparation

## 📋 Section Overview
- Load Spider Text2SQL dataset from HuggingFace
- Convert to Chat Completions format (required by Fireworks)
- Save as JSONL files for upload
- Alternative: Use GCS datasets directly

## 💡 GCS Alternative

If your data is already in Google Cloud Storage, you can skip data preparation and use:

```bash
# Upload dataset directly from GCS (no local file needed)
firectl create dataset my-dataset-name \
  --external-url gs://your-bucket/path/to/dataset.jsonl
```

For this example, we'll prepare data locally to show the complete workflow.

In [ ]:
# Optional: Setup GCS Bucket Access (Skip if using local files only)
print("🔐 Setting up GCS bucket access for Fireworks...")
print("💡 This is only needed if you want to use the GCS upload method")

# For GCS datasets, Fireworks needs access to your bucket
# The commands below grant the necessary permissions
print("""
📋 To setup GCS access, run these gcloud commands:

# Grant access to Fireworks service accounts
gcloud storage buckets add-iam-policy-binding gs://your-bucket \\
  --member=serviceAccount:fireworks-control-plane@<your-project-here>.iam.gserviceaccount.com \\
  --role=roles/storage.objectViewer

gcloud storage buckets add-iam-policy-binding gs://your-bucket \\
  --member=serviceAccount:inference@your-project-here>.iam.gserviceaccount.com \\
  --role=roles/storage.objectViewer

📚 More info: https://docs.fireworks.ai/fine-tuning/fine-tuning-external-dataset
""")

## 📊 Load Spider Dataset

We'll use the Spider Text2SQL dataset: [xlangai/spider](https://huggingface.co/datasets/xlangai/spider).

This is a widely-used benchmark with 7,000 training examples for text-to-SQL generation.

In [ ]:
# Load Spider Text2SQL dataset from HuggingFace

from datasets import load_dataset
import pandas as pd


print("📊 Loading Spider dataset...")

dataset = load_dataset("spider", cache_dir="/tmp/hf_cache")

df = pd.DataFrame(dataset["train"])

print(f"✅ Dataset loaded: {len(df):,} examples")
print(f"📊 Columns: {list(df.columns)}")

# Show sample data
print("\n📋 Sample text-to-SQL examples:")
sample_data = df[['question', 'query']].head(3)
for idx, row in sample_data.iterrows():
    print(f"\n{idx + 1}. Question: {row['question']}")
    print(f"   SQL: {row['query']}")


# 3. Dataset Upload to Fireworks

## 📋 Section Overview
- Convert data to Chat Completions format (required by Fireworks)
- Create JSONL files for upload
- Upload using `firectl create dataset` command
- Show both local file and GCS upload methods

## 🔄 Convert to Chat Completions Format

Fireworks requires data in Chat Completions format with `system`, `user`, and `assistant` messages.

Each training example needs three parts:
- **System**: Instructions for the model
- **User**: The natural language question  
- **Assistant**: The expected SQL query response


In [ ]:
# Create example in Chat Completions format
import json

# System prompt for Text2SQL fine-tuning
SYSTEM_PROMPT = """You are a SQL query generator. Your task is to convert natural language questions into valid SQL queries.

Rules:
- Return ONLY the SQL query
- No explanations or comments
- No markdown code blocks or formatting
- No references to examples, repositories, or documentation
- No prefacing text like "Here's the query:" or "The SQL is:"

Just return the raw SQL query."""

# Example format
example = {
    "messages": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": df.iloc[0]["question"]},
        {"role": "assistant", "content": df.iloc[0]["query"]}
    ]
}

print("📋 Chat Completions format example:")
print(json.dumps(example, indent=2))

## 💾 Create JSONL Files

For this demo, we'll use 1,000 examples for faster training. You can use the full 7,000 examples for better results.

In [ ]:
# Convert dataset to JSONL format
from datetime import datetime
import random

# Create sampled dataset for faster training
sampled_df = df.sample(n=1000, random_state=42)

def df_to_jsonl(df, output_file):
    """Convert DataFrame to JSONL with Chat Completions format"""
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            chat_record = {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": row["question"]},
                    {"role": "assistant", "content": row["query"]}
                ]
            }
            f.write(json.dumps(chat_record) + '\n')
    print(f"✅ Saved {len(df)} examples to {output_file}")

# Create files with timestamp
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
full_file = f"spider_full_{timestamp}.jsonl"
sample_file = f"spider_1k_{timestamp}.jsonl"

# Save both datasets
df_to_jsonl(sampled_df, sample_file)
df_to_jsonl(df, full_file)

print(f"\n📄 Files created:")
print(f"   📋 Sample dataset: {sample_file}")
print(f"   📋 Full dataset: {full_file}")



## 📤 Upload Dataset - Local File Method

Upload your JSONL file to Fireworks:

```bash
firectl create dataset DATASET_NAME ./your-file.jsonl
```


In [ ]:
# Execute the dataset upload command
import subprocess
import sys

# Create unique dataset name
dataset_name = f"spider-text2sql-full-{timestamp}"

print(f"📤 Uploading dataset: {dataset_name}")
print(f"📄 File: {full_file}")

# Build and run the command
cmd = ["firectl",  "create", "dataset", dataset_name, sample_file]
print(f"🔧 Running: {' '.join(cmd)}")

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✅ Dataset upload successful!")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"❌ Upload failed: {e}")
    print(f"Error output: {e.stderr}")
except FileNotFoundError:
    print("❌ firectl not found. Make sure it's installed and in your PATH.")


In [ ]:
! firectl get dataset {dataset_name}

## 🌐 Alternative: Upload from GCS

If your data is already in Google Cloud Storage:

```bash
firectl create dataset DATASET_NAME --external-url gs://your-bucket/path/to/dataset.jsonl
```


In [ ]:
# The same transformed Text2Sql dataset "spider" is available in this public GCS bucket as well

! curl -s https://storage.googleapis.com/fireworks-demos/sft_sample_data/spider_original_full_7k.jsonl | head -c 200

dataset_name_gcs = dataset_name + "-gcs"
! firectl create dataset {dataset_name_gcs} --external-url gs://fireworks-demos/sft_sample_data/spider_original_full_7k.jsonl
print("Uploading dataset to Fireworks from a GCS Bucket: ", dataset_name_gcs)

# Check dataset exists
! firectl get dataset {dataset_name_gcs}

# 4. Create Fine-tuning Job

## 📋 Section Overview  
- Create fine-tuning job with `firectl create supervised-fine-tuning-job`
- Monitor training progress
- Best practices for parameters

## 🚀 Start Fine-tuning

Here's the key CLI command to start training:

```bash
firectlcreate supervised-fine-tuning-job \
  --base-model accounts/fireworks/models/qwen2p5-coder-32b-instruct \
  --dataset DATASET_NAME \
  --output-model MODEL_NAME \
  --display-name "Text2SQL Fine-tune" \
  --epochs 3 \
  --learning-rate 0.0001 \
  --turbo \
  --early-stop \
  --eval-auto-carveout
```

## 📊 Parameter Guide

| Parameter | Description | Recommended Value |
|-----------|-------------|-------------------|
| `--base-model` | Base model to fine-tune | `qwen2p5-coder-32b-instruct` |
| `--dataset` | Your uploaded dataset ID | From step 3 |
| `--output-model` | Name for your fine-tuned model | `my-text2sql-model` |
| `--epochs` | Training iterations | `3` (start small) |
| `--learning-rate` | Learning rate | `0.0001` |
| `--turbo` | Faster training | Always include |
| `--early-stop` | Prevent overfitting | Always include |
| `--eval-auto-carveout` | Auto validation split | Always include |

In [ ]:
# Execute fine-tuning job creation
import subprocess
import re

# Create unique model name
model_name = f"spider-sql-{timestamp}"
display_name = f"Text2SQL Fine-tune {timestamp}"

print("🚀 Creating fine-tuning job...")
print(f"📋 Dataset: {dataset_name}")
print(f"📋 Output Model: {model_name}")

# Build the fine-tuning command
cmd = [
    "firectl",
    "create", "supervised-fine-tuning-job",
    "--base-model", "accounts/fireworks/models/qwen2p5-coder-32b-instruct",
    "--dataset", dataset_name,
    "--output-model", model_name,
    "--display-name", display_name,
    "--epochs", "3",
    "--learning-rate", "0.0001",
    "--turbo",
    "--early-stop",
    "--eval-auto-carveout"
]

print(f"🔧 Running: {' '.join(cmd)}")

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✅ Fine-tuning job created successfully!")
    print(result.stdout)

    # Extract job ID from output
    pattern = r"Name: accounts/[^/]+/supervisedFineTuningJobs/([a-zA-Z0-9]+)"

    match = re.search(pattern, result.stdout)
    if match:
        job_id = match.group(1)
        print(f"🎯 Job ID: {job_id}")
        print(f"💡 Save this: export JOB_ID='{job_id}'")
        print(f"\n🔍 Monitor with: firectl get supervised-fine-tuning-job {job_id}")
    else:
        print("💡 Job created but couldn't extract ID. Check the output above.")

except subprocess.CalledProcessError as e:
    print(f"❌ Fine-tuning job creation failed: {e}")
    print(f"Error output: {e.stderr}")
    if "dataset" in str(e.stderr).lower():
        print("💡 Make sure your dataset was uploaded successfully first!")
except FileNotFoundError:
    print("❌ firectl not found. Make sure it's installed and in your PATH.")


# 5. Monitor Training Progress

## 📋 Section Overview
- Check job status with CLI commands
- Monitor training metrics
- Know when training is complete

## 📊 Monitoring Commands

Check your job status with these commands:

```bash
# List all jobs
firectl list supervised-fine-tuning-jobs

# Check specific job
firectl get supervised-fine-tuning-job JOB_ID

# Check your models (after completion)
firectl list model
```

In [ ]:
# Execute monitoring commands
import subprocess

print("📊 Monitoring your fine-tuning job...")

try:
    cmd = ["firectl", "get", "supervised-fine-tuning-job", job_id]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✅ Job details:")
    print(result.stdout)

    # Check if job is completed
    if "JOB_STATE_COMPLETED" in result.stdout:
        print("🎉 Job completed! Ready for deployment.")
    elif "JOB_STATE_RUNNING" in result.stdout:
        print("⏳ Job is still running...")
    elif "JOB_STATE_VALIDATING" in result.stdout:
        print("🔍 Job is validating...")
    elif "JOB_STATE_QUEUED" in result.stdout:
        print("📋 Job is queued...")
    else:
        print("📊 Check the status above for current state.")

except subprocess.CalledProcessError as e:
    print(f"❌ Failed to check job {job_id}: {e}")
    print(f"Error: {e.stderr}")
except FileNotFoundError:
    print("❌ firectl not found.")


## 🚀 Deploy Your Model

Once training is **JOB_STATE_COMPLETED**, deploy your model:

```bash
firectl create deployment accounts/pyroworks/models/MODEL_NAME --enable-addons
```

This automatically merges the LoRA weights into the base model you trained on.

In [ ]:
# Execute model deployment
import subprocess
import re

print("🚀 Deploying your fine-tuned model...")

# Build deployment command (using the model name from earlier)
model_path = f"accounts/pyroworks/models/{model_name}"
# Manually set your model path - demo purposes
model_path = "accounts/pyroworks/models/spider-sql-finetuned" # accounts/fireworks/models/qwen2p5-coder-7b

cmd = ["firectl",  "create", "deployment", model_path, "--enable-addons"]

print(f"🔧 Running: {' '.join(cmd)}")
print("⏳ Note: This will only work if your fine-tuning job has COMPLETED")

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✅ Model deployment successful!")
    print(result.stdout)

    # Extract deployment ID
    pattern = r"Name: accounts/[^/]+/deployments/([a-zA-Z0-9]+)"
    match = re.search(pattern, result.stdout)
    if match:
        deployment_id = match.group(1)
        print(f"🎯 Deployment ID: {deployment_id}")
        print(f"💡 Save this: export DEPLOYMENT_ID='{deployment_id}'")

except subprocess.CalledProcessError as e:
    print(f"❌ Deployment failed: {e}")
    print(f"Error output: {e.stderr}")
    if "not found" in str(e.stderr):
        print("💡 Make sure your fine-tuning job completed successfully first!")
except FileNotFoundError:
    print("❌ firectl not found. Make sure it's installed and in your PATH.")


In [ ]:
! firectl get deployment {deployment_id}

### Use load LoRA instead for hosting multiple loras on the same endpoint
* https://fireworks.ai/docs/fine-tuning/multi-lora

Create a new base model deployment
```bash
firectl create deployement accounts/fireworks/models/qwen2p5-coder-7b-instruct
```

Load LoRA into the base deployment
```bash
firectl load-lora {model_path} --deployment="<deployment-id>"
```

Load another LoRA into the base deployment
```
firectl load-lora {model_path_path_2} --deployment="<deployment-id>"
```

# 7. Test Your Model

## 🧪 Query with curl

Test your deployed model from the command line:

```bash
curl -X POST https://api.fireworks.ai/inference/v1/chat/completions \
  -H "Authorization: Bearer $FIREWORKS_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{
    "model": "accounts/pyroworks/models/spider-sql-sft20250722-115217,
    "messages": [
      {"role": "system", "content": "You are a SQL query generator. Return ONLY the SQL query."},
      {"role": "user", "content": "What is the average salary by department?"}
    ],
    "temperature": 0,
    "max_tokens": 200
  }'
```


In [ ]:
# Test your fine-tuned model
import requests
import os

print("🧪 Testing your fine-tuned model...")

# Model URL (using model name from earlier)
url = "https://api.fireworks.ai/inference/v1/chat/completions"

# Test questions
test_questions = [
    "What is the average salary by department?",
    "How many employees are in each department?",
    "Who are the top 3 highest paid employees?"
]

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

print(f"🎯 Testing model: {model_path}")
print("⏳ Note: Model must be deployed first!")

for i, question in enumerate(test_questions, 1):
    print(f"\n📋 Test {i}: {question}")

    data = {
        "model": model_path,
        "messages": [
            {"role": "system", "content": "You are a SQL query generator. Return ONLY the SQL query."},
            {"role": "user", "content": question}
        ],
        "temperature": 0,
        "max_tokens": 200
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()

        result = response.json()
        sql_query = result['choices'][0]['message']['content'].strip()

        print(f"🔧 Generated SQL: {sql_query}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")
        break

print("\n✅ Testing complete!")


# 🎉 Summary

Congratulations! You've completed the full Fireworks AI fine-tuning workflow.

## 📋 What You Accomplished

1. ✅ **Setup**: Installed and authenticated with Fireworks CLI
2. ✅ **Data Prep**: Loaded Spider dataset and converted to Chat Completions format  
3. ✅ **Upload**: Created dataset in Fireworks using `firectl create dataset`
4. ✅ **Fine-tune**: Started training job with `firectl create supervised-fine-tuning-job`
5. ✅ **Monitor**: Checked job status with `firectl get supervised-fine-tuning-job`
6. ✅ **Deploy**: Deployed model with `firectl create deployment`
7. ✅ **Test**: Queried your custom Text2SQL model

## 🔧 Key Commands to Remember

```bash
# Core workflow
firectl create dataset NAME ./data.jsonl
firectl create supervised-fine-tuning-job --dataset NAME --output-model MODEL
firectl create deployment accounts/ACCOUNT/models/MODEL --enable-addons

# Monitoring  
firectl list supervised-fine-tuning-jobs
firectl get supervised-fine-tuning-job JOB_ID
firectl list deployments
firectl get deployment DEPLOYMENT_ID
```

## 🚀 Next Steps

- Try different base models (`qwen2p5-coder-7b`, `llama-v3p1-8b-instruct`)
- Experiment with more training data (use the full 7k dataset)
- Adjust hyperparameters (`--epochs`, `--learning-rate`)
- Deploy multiple models for A/B testing

## 📚 Resources

- [Fireworks Fine-tuning Docs](https://docs.fireworks.ai/fine-tuning/fine-tuning)
- [firectl CLI Reference](https://docs.fireworks.ai/tools-sdks/firectl/firectl)
- [Chat Completions Format](https://docs.fireworks.ai/guides/querying-text-models)


# Standalone Duck DB Demo

In [ ]:
#! pip install duckdb
import requests
import os
import json
import duckdb
import pandas as pd
from datetime import datetime, date

# Create a demo DuckDB database
def create_demo_database():
    """Create a demo database with sample data using DuckDB"""
    conn = duckdb.connect(':memory:')  # In-memory database for demo

    # Create tables
    conn.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name VARCHAR NOT NULL,
            department VARCHAR NOT NULL,
            salary DOUBLE NOT NULL,
            hire_date DATE NOT NULL
        )
    ''')

    conn.execute('''
        CREATE TABLE departments (
            id INTEGER PRIMARY KEY,
            name VARCHAR NOT NULL,
            location VARCHAR NOT NULL
        )
    ''')

    # Insert sample data
    employees_data = [
        (1, 'Alice Johnson', 'Engineering', 85000, '2022-01-15'),
        (2, 'Bob Smith', 'Marketing', 72000, '2021-08-20'),
        (3, 'Carol Davis', 'Engineering', 92000, '2020-03-10'),
        (4, 'David Wilson', 'Sales', 68000, '2022-11-05'),
        (5, 'Eva Brown', 'Marketing', 75000, '2021-12-01'),
        (6, 'Frank Miller', 'Engineering', 88000, '2020-07-22'),
        (7, 'Grace Lee', 'Sales', 72000, '2022-05-18'),
        (8, 'Henry Taylor', 'HR', 65000, '2021-09-30')
    ]

    departments_data = [
        (1, 'Engineering', 'San Francisco'),
        (2, 'Marketing', 'New York'),
        (3, 'Sales', 'Chicago'),
        (4, 'HR', 'Austin')
    ]

    conn.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', employees_data)
    conn.executemany('INSERT INTO departments VALUES (?, ?, ?)', departments_data)

    return conn

# Create the demo database
print("Creating demo database with DuckDB...")
db_conn = create_demo_database()

# Show the data
print("\nSample data:")
print("Employees:")
employees_df = db_conn.execute("SELECT * FROM employees").df()
print(employees_df)

print("\nDepartments:")
departments_df = db_conn.execute("SELECT * FROM departments").df()
print(departments_df)

# Get the schema for the API
schema_info = db_conn.execute("DESCRIBE employees").df()
schema = "CREATE TABLE employees (\n"
for _, row in schema_info.iterrows():
    schema += f"  {row['column_name']} {row['column_type']},\n"
schema = schema.rstrip(',\n') + "\n);\n\n"

schema_info = db_conn.execute("DESCRIBE departments").df()
schema += "CREATE TABLE departments (\n"
for _, row in schema_info.iterrows():
    schema += f"  {row['column_name']} {row['column_type']},\n"
schema = schema.rstrip(',\n') + "\n);"

print(f"\nDatabase Schema:")
print(schema)


In [ ]:
# Fireworks Base URL
import re
url = "https://api.fireworks.ai/inference/v1/chat/completions"

# The latest fine-tune you created
model = model_path


### These are just examples for demo purposes. Your account won't have these exact deployments unless you create them.
# Model URL - change to your latest model you just created. This one I created earlier.
# qwen2p5-coder-32b-instruct
base_qc32b = "accounts/fireworks/models/qwen2p5-32b-instruct#accounts/pyroworks/deployments/l5nzg8f2" # base model
ft_qc32b = "accounts/pyroworks/models/spider-sql-sft20250722-115217" # fine-tuned variant

# qwen2p5-coder-7b
base_qc7b = "accounts/fireworks/models/qwen2p5-coder-7b-instruct#accounts/pyroworks/deployments/tqn20ka9" # base model
ft_qc7b = "accounts/pyroworks/models/spider-sql-finetuned" # fine-tuned variant
model = ft_qc7b

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Example questions to test
questions = [
    "What is the average salary by department?",
    "How many employees are in each department?",
    "Who are the top 3 highest paid employees?",
    "What is the total salary budget for Engineering?",
    "What location has the most employees?",
    "Who is the most tenured employee in Marketing?",
    "Who is the most tenured employee in Marketing and where do they live?",
    "Where does Eva Brown live and who does she work with?",
    "Which employees started the same year together?",
    "Which two employees started within the fewest days of each other?",
    "Does engineering or marketing have more people?"
]

# The fine-tuned 7b variant does better on the compound queries

total_correct = 0
for question in questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print(f"{'='*60}")

    # Prepare the chat messages
    data = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": f"""You are a SQL query generator. Your task is to convert natural language questions into valid SQL queries.

                Rules:
                - Return ONLY the SQL query
                - No explanations or comments
                - No markdown code blocks or formatting
                - No references to examples, repositories, or documentation
                - No prefacing text like "Here's the query:" or "The SQL is:"

                Just return the raw SQL query."""
            },
            {
                "role": "user",
                "content": f"Database schema:\n{schema}\n\nQuestion: {question}"
            }
        ],
        "temperature": 0,
        "max_tokens": 500
    }

    # Make the request
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()

        # Parse the response
        result = response.json()
        sql_query = result['choices'][0]['message']['content'].strip()

        print("Generated SQL:")
        print(sql_query)

        # Execute the query on our demo database
        try:
            result_df = db_conn.execute(sql_query).df()
            print("\nQuery Result:")
            print(result_df)
            total_correct += 1
        except Exception as e:
            print(f"\nError executing query: {e}")

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")


# Clean up
db_conn.close()
print(f"\n{'='*60}")
print("Demo completed!")

model_name = re.search(r"models/([^#]+)", model).group(1)
print(f" {model_name} completion percentage: {total_correct / len(questions) * 100:.2f}%")

data = {}